# Bouncing Ball Example

_See also [this report](http://msdl.cs.mcgill.ca/people/slacoste/research/report/SummerReport.html)._

A basic use-case to illustrate the usefulness of multimodelling is the bouncing ball example. In this example, a ball is dropped or thrown vertically towards the ground. When it hits the ground, it bounces due to its elasticity. The ball can get stuck on the ground after a while (say there is a stair preventing the ball to bounce up). There are three states in this system:
1. **Free Fall:** The ball is only influenced by gravity and falls towards the ground. Here, the height of the ball $y$ and its velocity $v$ can be given as shown below. $y_0$ is the initial height, $v_0$ its initial velocity, $t$ the simulation time and $g$ the gravitational acceleration. If $v$ is positive, the ball moves upwards. Otherwise, the ball moves downwards.
$$y = y_0 + v_0 \cdot t - \dfrac{g\cdot t^2}{2}$$
$$ v = v_0 - g \cdot t$$<br/>
1. **Bounce:** The ball bounces on the ground and starts moving upwards again. Its action is identified by $v_{new} = -k \cdot v$, with $k$ the coefficient of restitution of the ball (if $k < 0$, the collisions will be inelastic; when $k$ equals 1, they are elastic; and if $k > 1$, they are superelastic). $v_{new}$ and $v$ respectively represent the velocity before and after the bounce. This mode will be entered if the ball has a $y \leq y_{floor}$.
1. **Stuck:** The ball is being prevented from bouncing again. Whenever a bounce needs to happen after $t_{stuck}$, the ball will enter _stuck_ mode instead. Here, $y$ will always equal $y_{floor}$.

### The Driver
It is clear that _(1. Free Fall)_ can be modeled using CBDs, whereas the other modes are modeled into a `Driver` Atomic DEVS that controls the system. This `Driver` knows that after a time of $t_{floor}$, the ball must bounce.

In [1]:
from pypdevs.DEVS import AtomicDEVS
from pypdevs.infinity import INFINITY

class Driver(AtomicDEVS):
    def __init__(self, name, y0, v0, g=9.81, k=1.0, t_stuck=80.0, dt=0.1, y_floor=0.0):
        AtomicDEVS.__init__(self, name)
        
        self.g = g
        self.k = k
        self.t_stuck = t_stuck
        self.delta_t = dt
        self.y_floor = y_floor
        
        self.state = {
            "time_next_bounce": 0.0,  # -- [1]
            "y0": y0,
            "v0": v0,
            "time": 0.0
        }
        
        self.y_out = self.addOutPort("y_out")   # Outputs new y0
        self.v_out = self.addOutPort("v_out")   # Outputs new v0
        
    def timeAdvance(self):
        return self.state["time_next_bounce"]
    
    def intTransition(self):
        self.state["time"] += self.timeAdvance()
        
        if self.state["time"] >= self.t_stuck:
            self.state["v0"] = 0.0
            self.state["y0"] = self.y_floor
            self.state["time_next_bounce"] = self.delta_t         # -- [2]
            return self.state
                
        time = self.compute_time_until_next_bounce()
        v0 = self.state["v0"]
        v = v0 - (self.g * time)
        self.state["v0"] = -self.k * v  # -- [3]
        self.state["y0"] = self.y_floor
        self.state["time_next_bounce"] = time
        return self.state
    
    def outputFnc(self):
        return {
            self.y_out: self.state["y0"],
            self.v_out: self.state["v0"]
        }
    
    def compute_time_until_next_bounce(self):
        v0 = self.state["v0"]
        y0 = self.state["y0"]
        return (v0 + ((v0 ** 2) + (2 * self.g * (y0 - self.y_floor))) ** 0.5) / self.g   # -- [4]


In the `Driver` class above, there are a few notes to be made (numbered in the comments), which refer to the points listed below:
1. The CBD model executes at time 0 already; **however**, the `Driver` needs to compute its next state.
1. We're stuck. We do not need to recompute the $y$-value(s).
1. The values for the next bounce are computed beforehand for simplicity. Here, the bounce formula is used to make the ball move upwards again.
1. The time at which the next bounce happens can be identified by solving the first equation of the _Free Fall_ mode to $t$.

### Free Fall
The CBD model for the _Free Fall_ mode is given below. Basically, this just implements the given equations.


In [2]:
from CBD.Core import *
from CBD.lib.std import *


class FreeFall(CBD):
    def __init__(self, block_name, g=(9.81), dt=0.1):
        CBD.__init__(self, block_name, input_ports=['v0', 'y0'], output_ports=['y'])
        
        self.addBlock(ConstantBlock("a", -g))
        self.addBlock(ConstantBlock("dt", dt))
        self.addBlock(IntegratorBlock("v"))
        self.addBlock(IntegratorBlock("x"))
        self.addConnection("a", "v")
        self.addConnection("v", "x")
        self.addConnection("x", "y")
        self.addConnection("dt", "v", input_port_name='delta_t')
        self.addConnection("dt", "x", input_port_name='delta_t')
        self.addConnection("v0", "v", input_port_name='IC')
        self.addConnection("y0", "x", input_port_name='IC')

        # Create the Blocks
#         self.addBlock(TimeBlock("time"))
#         self.addBlock(ProductBlock("square"))
#         self.addBlock(ConstantBlock("c", value=(-g / 2)))
#         self.addBlock(ProductBlock("a_mul"))
#         self.addBlock(ProductBlock("v_mul"))
#         self.addBlock(AdderBlock("y_sum"))
#         self.addBlock(AdderBlock("sum_total"))

#         # Create the Connections
#         self.addConnection("y0", "y_sum", input_port_name='IN1')
#         self.addConnection("v0", "v_mul", input_port_name='IN1')
#         self.addConnection("time", "square", output_port_name='relative', input_port_name='IN1')
#         self.addConnection("time", "square", output_port_name='relative', input_port_name='IN2')
#         self.addConnection("time", "v_mul", output_port_name='relative', input_port_name='IN2')
#         self.addConnection("c", "a_mul", output_port_name='OUT1', input_port_name='IN2')
#         self.addConnection("square", "a_mul", output_port_name='OUT1', input_port_name='IN1')
#         self.addConnection("v_mul", "y_sum", output_port_name='OUT1', input_port_name='IN2')
#         self.addConnection("y_sum", "sum_total", output_port_name='OUT1', input_port_name='IN1')
#         self.addConnection("a_mul", "sum_total", output_port_name='OUT1', input_port_name='IN2')
#         self.addConnection("sum_total", "y", output_port_name='OUT1')
        
        self.addFixedRateClock("clock", dt)
        

### Data Collection
Now, all that's needed is a way of plotting the data. The `PointCollector` below will collect all data points from the CBD, whereas we will use the `CBD.realtime.plotting` module for displaying the data.

In [3]:
from pypdevs.DEVS import AtomicDEVS

class PointCollector(AtomicDEVS):
    def __init__(self, name):
        AtomicDEVS.__init__(self, name)
        
        self.state = {
            "data": [],
            "time": 0.0
        }
        
        self.input = self.addInPort("input")
        
    def extTransition(self, inputs):
        self.state["time"] += self.elapsed
        if self.input in inputs:
            value = inputs[self.input]
            if self.elapsed == 0 and len(self.state["data"]) > 0:
                self.state["data"].pop()  # -- [1]
            self.state["data"].append((self.state["time"], value))
        return self.state
    
    @property
    def data_xy(self):  # -- [2]
        data = self.state["data"][:]  # -- [3]
        return [x for x, _ in data], [y for _, y in data]

With the following remarks:
1. We only keep track of the last data point when multiple happen at the same time.
1. This property will be used by the `CBD.realtime.plotting` module to obtain the data for the plot.
1. To ensure thread safety of this call, the data is copied first. Alternatively, a locking mechanism may be used instead.

### Putting it all Together
By making use of the `CBDRunner` class, a CBD model can be encapuslated inside a DEVS model.

In [4]:
# import sys, os
# sys.path.insert(0, os.path.join(os.getcwd(), "..", "..", "src"))  # -- [1]

from CBD.converters.pypdevs import CBDRunner
from pypdevs.DEVS import CoupledDEVS

class BouncingBall(CoupledDEVS):
    def __init__(self, y0, v0, g=9.81, k=1.0, t_stuck=80.0, y_floor=0.0, dt=0.1):
        CoupledDEVS.__init__(self, "BouncingBall")
        
        self.driver    = self.addSubModel(Driver("BBDriver", y0, v0, g, k, t_stuck, dt, y_floor))
        self.freefall  = self.addSubModel(CBDRunner("FreeFall",
                                                    FreeFall("FreeFall", g, dt), {"y0": y0, "v0": v0}))
        self.collector = self.addSubModel(PointCollector("plot"))
        
        self.connectPorts(self.driver.y_out, self.freefall.inputs["y0"])
        self.connectPorts(self.driver.v_out, self.freefall.inputs["v0"])
        self.connectPorts(self.freefall.outputs["y"], self.collector.input)
        
    def select(self, imm_children):   # -- [2]
        if self.driver in imm_children:
            return self.driver
        return imm_children[0]

1. Ensure the `CBDRunner` class can be loaded.
1. If the `Driver` wants to execute, give it precedence over the `CBDRunner`.

Now, we add a plotter and start the simulation!

In [9]:
%matplotlib notebook

from CBD.realtime.plotting import PlotManager, ScatterPlot, follow
from pypdevs.simulator import Simulator
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.set_xlim((0, 100))
ax.set_ylim((0, 110))

ball = BouncingBall(100.0, 3.0, k=0.95, t_stuck=80.0, dt=0.1)
sim = Simulator(ball)
sim.setClassicDEVS()
sim.setRealTime()
# sim.setVerbose()
sim.setTerminationTime(90)

manager = PlotManager()
manager.register("ball", ball.collector, (fig, ax), ScatterPlot(s=2, c='red'))

sim.simulate()

plt.show()

<IPython.core.display.Javascript object>

In [10]:
manager.terminate()
# ball.freefall.state["CBD"].getSignalHistory('y')[-1]
# ball.collector.state["data"]